Dieses Skript soll aus der erstellten Übersicht von Inkar-indikatoren die einzelnen csv's für die jeweiligen Indikatoren erstellen..


In [2]:
import pandas as pd # type: ignore

In [4]:
# Liste der gesamten Indikatoren wird geladen

data = pd.read_csv("../../out/data.csv")
data.head()

,select,category,category_de,category_nl,name,name_de,algorithmus,description,statistical_basis,name_nl,source_de,source_id_de,source_nl,source_id_nl,gemeinden,kreise,zeitbezug_gemeinden,zeitbezug_kreise
0,NaN,NaN,Absolutzahlen,NaN,NaN,Arbeitslose,NaN,Es handelt sich um die Zahl der Arbeitslosen i...,Arbeitsmarktstatistik der Bundesagentur für Ar...,NaN,Inkar,2176,NaN,NaN,NaN,X,NaN,1995 - 2020
1,NaN,NaN,Absolutzahlen,NaN,NaN,Bruttoinlandsprodukt in 1000 Euro,NaN,Das Bruttoinlandsprodukt (BIP) misst die Produ...,"Arbeitskreis ""Volkswirtschaftliche Gesamtrechn...",NaN,Inkar,4273,NaN,NaN,NaN,X,NaN,2000 - 2020
2,NaN,NaN,Absolutzahlen,NaN,NaN,Sozialversicherungspflichtig Beschäftigte am W...,NaN,Es handelt sich um die Zahl der sozialversiche...,Beschäftigtenstatistik der Bundesagentur für A...,NaN,Inkar,8766,NaN,NaN,NaN,X,NaN,1997 - 2020
3,NaN,NaN,Absolutzahlen,NaN,NaN,Sozialversicherungspflichtig Beschäftigte am A...,NaN,Es handelt sich um die Zahl der sozialversiche...,Beschäftigtenstatistik der Bundesagentur für A...,NaN,Inkar,8774,NaN,NaN,NaN,X,NaN,1997 - 2020
4,NaN,NaN,Absolutzahlen,NaN,NaN,Bevölkerung (mit Korrektur VZ 1987/Zensus 2011),NaN,Zwischen der Zahl der Bevölkerung am 31.12.201...,"Bundesinstitut für Bau-, Stadt- und Raumforsch...",NaN,Inkar,10806,NaN,NaN,NaN,X,NaN,1995 - 2020


In [5]:
# Code um die ausgewählten Indicatoren zu filtern und in einer csv abzuspeichern, also eine indicator_list

indicators = pd.DataFrame(columns=['category_de', 'name_de', 'source_de', 'source_id_de', 'category_nl', 'name_nl', 'source_nl', 'source_id_nl'])

for index in data.index:
    if data['select'][index] == "x":
        print(index)
        category_value = data['category_de'][index]
        indicator_name = data['name_de'][index]
        source_value = data['source_de'][index]
        source_id_value = int(data['source_id_de'][index])
        indicators.loc[len(indicators)] = {'category_de': category_value, 'name_de': indicator_name, 'source_de': source_value, 'source_id_de':source_id_value}


indicators.reset_index(drop=True, inplace=True)
indicators



,category_de,name_de,source_de,source_id_de,category_nl,name_nl,source_nl,source_id_nl


In [8]:
#Liest den Inkar Datensatz ein

inkar = pd.read_csv("../data/inkar_2022.csv",sep=";",decimal=",",dtype={'Kennziffer':str,'Kennziffer_EU':str,'Name':str}, low_memory=False)
inkar.head()

,Bereich,ID,Indikator,Raumbezug,Kennziffer,Name,Zeitbezug,Wert
0,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1995,10494.0
1,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1996,11921.0
2,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1997,13213.0
3,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1998,13301.0
4,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1999,12048.0


In [9]:
# Legt die Selektion des Raums fest und filtert den Datensatz danach

raumbezug_selection = ["Kreise","Gemeinden", "Bund", "Bundesländer", "Gemeindeverbände"]
inkar = inkar[inkar['Raumbezug'].isin(raumbezug_selection)]

In [10]:
#Filternt den gesamten Inkardatensatz aufgrund der Kategorien, QuellenID und Name von den Indikatoren

filtered_inkar = inkar[(inkar['Bereich'].isin(indicators['category_de'])) & (inkar['ID'].isin(indicators['source_id_de'])) & (inkar['Indikator'].isin(indicators['name_de']))]
filtered_inkar.reset_index(drop=True, inplace=True)
filtered_inkar.head()

,Bereich,ID,Indikator,Raumbezug,Kennziffer,Name,Zeitbezug,Wert
0,Absolutzahlen,12586,Bevölkerung gesamt,Bund,111,Bundesrepublik Deutschland,1995,81817498.0
1,Absolutzahlen,12586,Bevölkerung gesamt,Bund,111,Bundesrepublik Deutschland,1996,82012160.0
2,Absolutzahlen,12586,Bevölkerung gesamt,Bund,111,Bundesrepublik Deutschland,1997,82057380.0
3,Absolutzahlen,12586,Bevölkerung gesamt,Bund,111,Bundesrepublik Deutschland,1998,82037011.0
4,Absolutzahlen,12586,Bevölkerung gesamt,Bund,111,Bundesrepublik Deutschland,1999,82163476.0


In [13]:
#Erstellt für jedem Indikator in der Indikator csv eine eigene csv mit den Werten (name mittlerweile veraltet, sollte eigentlich die indicator_id sein)

indicators_csv = filtered_inkar['Indikator'].unique()
wanted_columns = ["Kennziffer", "Wert", "Zeitbezug"]
for indikator in indicators_csv:
    
    currentData = filtered_inkar[filtered_inkar['Indikator'] == indikator]
    currentData = currentData[wanted_columns]
    #indikator = indikator.split(" ")[0];
    indikator = indikator.replace(" ", "")
    indikator = indikator.replace("<", "")
    indikator = indikator.replace(">", "")
    print(indikator)
    fileName = f"C:/Users/juliu/VSCODE/VSCODE/inkar-indicators/src/out/indicator_{indikator}.csv"
    currentData.to_csv(fileName, index=False)
    

Bevölkerunggesamt
ErwerbsfähigeBevölkerung(15bisunter65Jahre)
Erwerbstätige
GeburtenjungerMütter(20Jahre)
GeburtenältererMütter(=40Jahre)
HaushaltemitKindern
Studierende
Einwohnerdichte
